# ◎ 네이버 OpenAPI와 Jaccard 유사도를 이용한 제품 카테고리 분류
---

1. 데이터 불러오기 및 결측치 처리 
2. 실적(performance), 평가(submission)데이터 통합
3. 상품명 전처리(정제)
4. [1차 제품 분류] 네이버 OpenAPI를 이용한 제품 분류
5. [2차 제품 분류] 자카드 유사도를 기반으로한 제품 분류
6. 

In [1]:
import pandas as pd 
import numpy as np 
import re 
import os
import sys
import urllib.request
import json
from tabulate import tabulate

### ▶ 데이터 불러오기 및 결측치 처리
- [Step1] 데이터 불러오기  
      1) 2019년 실적 데이터 (performance)  
      2) 2020년 평가 데이터 (submission)


- [Step2] '취급액' 컬럼 Null값 처리  
      1) '상품군' 컬럼이 '무형'인 데이터 제거  
      2) '취급액'이 Null인 경우 '0'으로 대체  

In [19]:
performance = pd.read_excel('2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx', skiprows=1)
submission = pd.read_excel('2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx', skiprows=1)

def treatNA(df):
    df = df[df['상품군']!='무형']
    df['취급액'] = df['취급액'].fillna(0)
    df = df.fillna(method='pad')
    return df

performance = treatNA(performance)
submission = treatNA(submission)

display(performance.head(5))
display(submission.head(5))

C:\Users\heian\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,0.0
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,0.0


### ▶ 실적/평가 데이터 통합
- [Step1] 데이터 통합 (df_all)
      :2019년 실적 데이터 (performance) + 2020년 평가 데이터 (submission)

In [20]:
df_all = pd.concat([performance, submission]).reset_index()
df_all.head()

,index,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


### ▶ 제품명 전처리

- [Step1] 제품명 전처리 함수 생성
        - 초기값 상품명 중복 제거
        - 상품명 내 불필요한 단어 제거
        - 정제 후 상품명 중복 제거
        
|  | 불필요한 단어 유형 | 예시 |
|-|-|-|
| 1 | 결제 관련 정보 | 무이자, 일시불, 이자 등 |
| 2 | 제품명 앞뒤 괄호에 포함된 단어 | (40인치) (특상품) (초특가) 등 |
| 3 | 수량 및 단위 관련 정보 | ~패키지, ~종, 세트, n+n, 더블팩, 싱글팩 |

- [Step2] 실적/평가/실적+평가 데이터에 각각 적용하여 전처리 후 상품명현황 파악
- [Step3] 


In [22]:
def productnm_cleansing(df):
    # 상품명 중복 제거
    uniq_pname=df['상품명'].unique()
    print('\n<# of Unique pnameuct Name> : \t{}건'.format(len(uniq_pname)))
    np.set_printoptions(threshold=sys.maxsize)

    # 추후 조합을 위해 전처리 이전 제품명과 이후 제품명을 저장합니다.
    p_name = pd.DataFrame(data=uniq_pname, columns=['Before Process'])

    # 제품명에서 불필요한 요소 전처리
    ap=[]
    re_stop = re.compile("""\([가-힣]{1,5}\+[가-힣]{1,5}\)|\(?무이자\)?\s?|\(?일시불\)?\s?|\(?초특가\)?\s?|\(?무\)\s?|\(?유\)\s?|\(?일\)\s?|무료체험|
    |포함|국내[가-힣]+\s|무료설치|\s?신제품\s?|\s?패키지\s?|[0-9]+종|풀코디|set|SET|풀세트|[0-9]+세트|더블팩|싱글팩|[0-9]{1,2}\+[0-9]{1,2}|[0-9]{1,2}인용|[0-9]{1,2}박스|[0-9\.]{1,4}미터|[0-9\.]{1,4}kg|[0-9\.]{1,4}[kKgG]|[0-9]{1,3}[벌롤종단구대P개통병포미봉팩장gL매]|
    |\s[0-9\.]{1,4}[Mm]|\(.{1,10}\)$|^\(.?\)|^[0-9]{2,4}\s|\s+[0-9]{2,4}\s|[0-9]{2,4}년\s|[0-9]{2,4}년형\s|시즌[0-9]|[0-9]{1,3}\%|\(.{1,20}\)$|기본형|고급형|오리지널|[대중소大中小]형|.{1,10}by|,|\s?총[0-9\s]|^[가-힣a-zA-Z]{3,3}의|
    |S\/S|F\/W|f\/w|s\/s|[가-힣]+형\s|[슈퍼]{0,2}싱글|\s[SQK퀸킹]{1,2}\s|[SQK퀸킹]{1,2}$|[가-힣]{0,2}사이즈""")

    remnants = re.compile('[",g\+lL-]\s|ml|_|\sx|[\[\]\］!"#$%&\'()*+,./:;<=>?@\^_`{|}~-]|\s종$|\s[0-9]+\s|\s{2,}')

    for i in p_name['Before Process']:
        tmp = re_stop.sub(' ',i)
        tmp = tmp.strip()
        tmp = remnants.sub(' ',tmp)
        ap.append(tmp.strip())

    # 전처리된 제품명 중복제거
    p_name['After Process'] = ap
    p_name.to_html('tmp.html')

    ap = list(set(ap))

    display(p_name.head(5))
    
    return p_name

In [30]:
#따로 돌려서 전처리 후 고유 상품명 현황 파악한다
performance_prd_clean = productnm_cleansing(performance)
submission_prd_clean = productnm_cleansing(submission)
df_all_prd_clean = productnm_cleansing(df_all)


<# of Unique pnameuct Name> : 	1692건


,Before Process,After Process
0,테이트 남성 셀린니트3종,테이트 남성 셀린니트
1,테이트 여성 셀린니트3종,테이트 여성 셀린니트
2,오모떼 레이스 파운데이션 브라,오모떼 레이스 파운데이션 브라
3,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,PAT 남성 소프트 기모 릴렉스팬츠
4,보코 리버시블 무스탕,보코 리버시블 무스탕



<# of Unique pnameuct Name> : 	349건


,Before Process,After Process
0,잭필드 남성 반팔셔츠 4종,잭필드 남성 반팔셔츠
1,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,쿠미투니카 쿨 레이시 란쥬쉐이퍼 팬티
2,바비리스 퍼펙트 볼륨스타일러,바비리스 퍼펙트 볼륨스타일러
3,램프쿡 자동회전냄비,램프쿡 자동회전냄비
4,벨레즈온 심리스 원피스 4종 패키지,벨레즈온 심리스 원피스



<# of Unique pnameuct Name> : 	1998건


,Before Process,After Process
0,테이트 남성 셀린니트3종,테이트 남성 셀린니트
1,테이트 여성 셀린니트3종,테이트 여성 셀린니트
2,오모떼 레이스 파운데이션 브라,오모떼 레이스 파운데이션 브라
3,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,PAT 남성 소프트 기모 릴렉스팬츠
4,보코 리버시블 무스탕,보코 리버시블 무스탕


In [44]:
# 실적/평가/전체 데이터의 상품명 현황
only_performance =  set(performance_prd_clean['After Process'].tolist()) - set(submission_prd_clean['After Process'].tolist())
only_submission = set(submission_prd_clean['After Process'].tolist()) - set(performance_prd_clean['After Process'].tolist())
per_sub_intersection = set(submission_prd_clean['After Process'].tolist()) & set(performance_prd_clean['After Process'].tolist())
per_sub_union = set(df_all_prd_clean['After Process'].tolist())

print("-----------------실적/평가 데이터 전처리 후 고유 상품명 현황 --------------------")
print("\t[Unique 상품명] 전체 데이터에  존재:\t",len(set(df_all_prd_clean['After Process'].tolist())))
print("\t[Unique 상품명] 실적 데이터에만 존재:\t", len(only_performance),"/",len(set(performance_prd_clean['After Process'].tolist())))
print("\t[Unique 상품명] 평가 데이터에만 존재:\t", len(only_submission),"/",len(set(submission_prd_clean['After Process'].tolist())))
print("\t[Unique 상품명] 실적/평가에 둘다 존재:\t", len(per_sub_intersection))

-----------------실적/평가 데이터 전처리 후 고유 상품명 현황 --------------------
	[Unique 상품명] 전체 데이터에  존재:	 1514
	[Unique 상품명] 실적 데이터에만 존재:	 1221 / 1266
	[Unique 상품명] 평가 데이터에만 존재:	 248 / 293
	[Unique 상품명] 실적/평가에 둘다 존재:	 45


### ▶ [1차 제품 분류] 네이버 OpenAPI를 이용한 제품 분류

- [Step1] 전체 데이터(평가+실적) 상품명에 해당하는 카테고리 1차 부여  

- [Step2] 1차 과정에서 찾지 못한 카테고리 처리  
        : 'Jaccard_Similarity' 유사도 기반으로 유사한 상품명의 분류 가져옴  

In [43]:
# 네이버 검색 API를 이용하여 세부 카테고리를 검색합니다.
def naver_shop(keyword):
    client_id = "DWNxDn7nWmw_X5LC74yj"
    client_secret = "MFrEW9wCWI"
    keyword = keyword.strip()
    encText = urllib.parse.quote(keyword)
    url = "https://openapi.naver.com/v1/search/shop.json?query=" + encText + "&display=1" # json 결과
    # url = "https://openapi.naver.com/v1/search/shop.xml?query=" + encText # xml 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        jsonString = response_body.decode('utf-8')
        j = json.loads(jsonString)
        if int(j['total']):
            c1 = j["items"][0]['category1']
            c2 = j["items"][0]["category2"]
            c3 = j["items"][0]["category3"]
            c4 = j["items"][0]["category4"]
            keyword2=''
        else : # 검색결과가 없을경우 모델명(숫자+알파벳 구성)과 사이즈(주로 한자어)를 제거하고 뒤에서부터 2개의 어절만으로 검색 재시도
            p_code = re.compile('[0-9a-zA-Z]+$|[一-龥]+$')
            keyword2 = p_code.sub(' ',keyword)
            keyword2 = keyword2.strip()
            keyword2 = p_code.sub(' ',keyword2)
            keyword2 = ' '.join(keyword2.split()[-2:])
            
            encText = urllib.parse.quote(keyword2)
            url = "https://openapi.naver.com/v1/search/shop.json?query=" + encText + "&display=1"
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            response = urllib.request.urlopen(request)
            rescode = response.getcode()
            
            if(rescode==200):
                response_body = response.read()
                jsonString = response_body.decode('utf-8')
                j = json.loads(jsonString)
                if int(j['total']):
                    c1 = j["items"][0]['category1']
                    c2 = j["items"][0]["category2"]
                    c3 = j["items"][0]["category3"]
                    c4 = j["items"][0]["category4"]
                else :
                    print(keyword,'->',keyword2)
                    print('>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<')
                    c1, c2, c3, c4 = '', '', '', ''
            else :
                print("Error Code:" + rescode)
    else:
        print("Error Code:" + rescode)
    return [keyword,c1,c2,c3,c4,keyword2]


In [45]:
from tqdm import tqdm_notebook

cat_df = pd.DataFrame(columns = ['pname','cat1','cat2','cat3','cat4','pname2'])

for item in tqdm_notebook(per_sub_union):
#     print(item)
    ns = naver_shop(item)
    cat_df=cat_df.append(pd.DataFrame([ns],columns=['pname','cat1','cat2','cat3','cat4','pname2']), ignore_index=True)

cat_df

에트로 프로푸미 웨이스트백 -> 프로푸미 웨이스트백
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
뱅뱅 여성 간절기 데님팬츠  KQE251 KQE253 KQE255 -> 데님팬츠 KQE251
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
뽕셰프 이봉원 특갈비탕 -> 이봉원 특갈비탕
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
크리스티나앤코 이지웨어세트 -> 크리스티나앤코 이지웨어세트
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
남해바다 손질왕꼬막살세트 -> 남해바다 손질왕꼬막살세트
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
USPA화이트라벨 여성트랙수트 -> USPA화이트라벨 여성트랙수트
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
황제진액 철갑상어 -> 황제진액 철갑상어
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
창녕 마늘양파듬뿍삼계탕 -> 창녕 마늘양파듬뿍삼계탕
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
메이듀 남성 린넨 블렌디드 슬립온 -> 블렌디드 슬립온
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
더커진거창특등급사과 -> 더커진거창특등급사과
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
삼성5도어냉장고 T9000 RF84R9203S8   시카고커트러리5p세트 -> RF84R9203S8 시카고커트러리5p세트
>>>>>>>>>>>>>>>>>>>>Search Failed!<<<<<<<<<<<<<<<<<<<<
비가린 배추김치 -> 

,pname,cat1,cat2,cat3,cat4,pname2
0,시크릿뮤즈 커버팩트,화장품/미용,베이스메이크업,파우더,팩트파우더,
1,마르엘라로사티 휘메일 풀스킨 밍크 후드코트,패션의류,여성의류,코트,,
2,오델로 남성 겨울모자,패션잡화,모자,비니,,남성 겨울모자
3,The 귀한 치마살,식품,축산,쇠고기,수입산쇠고기,
4,파격찬스 굿프렌드 스트레칭 안마 매트 GOOD A7,생활/건강,안마용품,안마매트,,
...,...,...,...,...,...,...
1509,보몽드 엘사 자수 쿨 시어서커 침구세트 SK,가구/인테리어,침구세트,이불베개세트,더블/퀸이불베개세트,시어서커 침구세트
1510,참바다손질낙지 양념장 연포탕육수,식품,수산,해산물/어패류,낙지,
1511,삼성 UHD TV UN65RU7150FXKR,디지털/가전,영상가전,TV,LEDTV,
1512,푸마 INNO 심리스 롱드로즈,패션의류,남성언더웨어/잠옷,팬티,,


In [49]:
#1차 제품 분류 값 파일로 저장
cat_df.to_csv('fisrt_search_cat_submission_hyein0908.csv')
cat_df.head()

,pname,cat1,cat2,cat3,cat4,pname2
0,시크릿뮤즈 커버팩트,화장품/미용,베이스메이크업,파우더,팩트파우더,
1,마르엘라로사티 휘메일 풀스킨 밍크 후드코트,패션의류,여성의류,코트,,
2,오델로 남성 겨울모자,패션잡화,모자,비니,,남성 겨울모자
3,The 귀한 치마살,식품,축산,쇠고기,수입산쇠고기,
4,파격찬스 굿프렌드 스트레칭 안마 매트 GOOD A7,생활/건강,안마용품,안마매트,,


In [50]:
len(cat_df)
unknown_ratio = len(cat_df[cat_df['cat1']==''])/len(cat_df)*100
print("***1차 분류에서 알아내지 못한 카테고리 비율 : ", round(unknown_ratio,3),"%")

***알아내지 못한 카테고리 비율 :  2.84 %


### ▶ [2차 제품 분류] 자카드 유사도를 기반으로한  제품 분류
 : 1차 분류에 실패한 상품명을 자카드 유사도를 기반으로 가장 높은 유사도를 가진 상품명의 제품분류를 가져오는 방식

---

#### > HOW? : Jaccard_Similarity

 : 문장을 명사단위로 쪼갠 뒤 두 문장간 (겹치는 명사의 개수) / (총 명사의 수) 로 구하는 유사도

-  예시) Jaccard('특대형 한방닭백숙', '김개똥의 닭백숙 세트') 
      
      --> ('특','특대형','한방','닭','닭백숙','백숙'), ('김개똥','닭','닭백숙','백숙','세트)   
      --> A&B = 6개(3쌍) ('닭','백숙','닭백숙')  
      --> A|B = 11개  

> Jaccard_sim = 6/11 = 54.5%


In [48]:
from tqdm import tqdm_notebook
from konlpy.tag import Kkma
kkma=Kkma()

def jaccard_sim(doc1, doc2):
    doc1 = kkma.nouns(doc1)
    doc2 = kkma.nouns(doc2)
    u_doc1 = set(doc1)
    u_doc2 = set(doc2)
    return len(u_doc1 & u_doc2) / len(u_doc1 | u_doc2)

In [51]:
# cat_df = pd.read_csv('좀더_전처리_카테고리결과.csv',index_col=0)
unknown = cat_df[cat_df['cat1']=='']
known = cat_df[cat_df['cat1']!='']
for i in tqdm_notebook(range(len(unknown))):
    best_sim = 0
    print(unknown.iloc[i].pname)
    for j in tqdm_notebook(range(len(known))):
        sim = jaccard_sim(unknown.iloc[i].pname,known.iloc[j].pname)
        if sim > best_sim :
            best_sim = sim
            best_ind = j
            print(best_sim)
    unknown.iloc[i].cat1 = known.iloc[best_ind].cat1
    unknown.iloc[i].cat2 = known.iloc[best_ind].cat2
    unknown.iloc[i].cat3 = known.iloc[best_ind].cat3
    unknown.iloc[i].cat4 = known.iloc[best_ind].cat4
    print(known.iloc[best_ind].cat1, known.iloc[best_ind].cat2,known.iloc[best_ind].cat3,known.iloc[best_ind].cat4)

에트로 프로푸미 웨이스트백


0.09090909090909091
0.18181818181818182
0.5555555555555556

패션잡화 여성가방 크로스백 
뱅뱅 여성 간절기 데님팬츠  KQE251 KQE253 KQE255


0.07142857142857142
0.14285714285714285
0.2857142857142857
0.36363636363636365
0.4

패션의류 여성의류 청바지 
뽕셰프 이봉원 특갈비탕


0.08333333333333333
0.1

식품 냉동/간편조리식품 즉석국/즉석탕 
크리스티나앤코 이지웨어세트


0.07692307692307693
0.18181818181818182
0.38461538461538464
0.5
0.5454545454545454

패션의류 여성의류 코디세트 
남해바다 손질왕꼬막살세트


...

0.1
0.5

식품 김치 포기김치 
AAE LA갈비


0.16666666666666666

식품 축산 축산가공식품 쇠고기양념육
마르엘라로사티 에코무스탕


0.35714285714285715
0.5
0.5555555555555556

패션의류 여성의류 청바지 
아리스토우 남성기모티셔츠


0.09090909090909091
0.16666666666666666
0.2857142857142857
0.3333333333333333
0.5714285714285714

패션의류 남성의류 티셔츠 
크리스티나앤코 기모밴딩팬츠


0.07692307692307693
0.09090909090909091
0.13333333333333333
0.38461538461538464
0.5
0.6363636363636364

패션의류 여성의류 바지 



In [52]:
#상품명-분류 테이블 결과 값 연결 

#정제 전 상품명-처리 후 상품명-제품 분류값 연결
tmp = pd.merge(p_name,pd.concat([known,unknown]),left_on = 'After Process', right_on = 'pname').drop(['pname','pname2','After Process'],axis=1)

#실적/평가 테이블과 조인
re_performance = pd.merge(performance,tmp,left_on='상품명',right_on='Before Process').drop(['Before Process'],axis=1)
re_performance.to_csv('categorized_performance_0908.csv')

re_submission = pd.merge(submission,tmp,left_on='상품명',right_on='Before Process').drop(['Before Process'],axis=1)
re_submission.to_csv('categorized_submission_0908.csv')

In [53]:
display(re_performance.head())
display(re_submission.head())

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,cat1,cat2,cat3,cat4
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,패션의류,남성의류,니트/스웨터,
1,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,패션의류,남성의류,니트/스웨터,
2,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,패션의류,남성의류,니트/스웨터,
3,2019-01-03 00:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,7329000.0,패션의류,남성의류,니트/스웨터,
4,2019-01-03 00:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,10481000.0,패션의류,남성의류,니트/스웨터,


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,cat1,cat2,cat3,cat4
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0,패션의류,남성의류,셔츠/남방,
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0,패션의류,남성의류,셔츠/남방,
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,0.0,패션의류,남성의류,셔츠/남방,
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,0.0,패션의류,여성언더웨어/잠옷,보정속옷,올인원/바디쉐이퍼
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,0.0,패션의류,여성언더웨어/잠옷,보정속옷,올인원/바디쉐이퍼


In [55]:
by_performance = pd.read_csv('categorized_performance_hyein.csv')
by_union = pd.read_csv('categorized_performance_0908.csv')

display(by_performance.isnull().sum())
display(by_union.isnull().sum())

Unnamed: 0          0
방송일시                0
노출(분)               0
상품명                 0
상품군                 0
판매단가                0
취급액                 0
방송일시_yymmdd         0
방송일시_hhmmss         0
seconds             0
방송일시_hhmm           0
ratings_mean        0
cat1                0
cat2                0
cat3              498
cat4            14159
dtype: int64

Unnamed: 0        0
방송일시              0
노출(분)             0
마더코드              0
상품코드              0
상품명               0
상품군               0
판매단가              0
취급액               0
cat1              0
cat2              0
cat3            462
cat4          14039
dtype: int64

In [59]:
by_union['same'] = '9999'
## performance로만 진행한 제품분류와 얼마나 틀린지 체크 !
by_performance[['cat1','cat2', 'cat3', 'cat4']]= by_performance[['cat1','cat2', 'cat3', 'cat4']].fillna('NA')
by_union[['cat1','cat2', 'cat3', 'cat4']] = by_union[['cat1','cat2', 'cat3', 'cat4']].fillna('NA')

for i in tqdm_notebook(range(len(by_performance))):
    if by_performance.cat1[i] != by_union.cat1[i]: 
        by_union['same'][i] = 'cat1'
    elif by_performance.cat2[i] != by_union.cat2[i]:
        by_union['same'][i] = 'cat2'
    elif by_performance.cat3[i] != by_union.cat3[i]:
        by_union['same'][i] = 'cat3'        
    elif by_performance.cat4[i] != by_union.cat4[i]:
        by_union['same'][i] = 'cat4'
    else: 
        by_union['same'][i] = 'equal'

C:\Users\heian\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\heian\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\heian\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\heian\Anaconda3\lib\site-packages\ipyke

In [60]:
by_union['same'].value_counts()

equal    36651
cat3       322
cat4       226
cat1       137
cat2        36
Name: same, dtype: int64